# No modificar, todo funciona

In [21]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chisquare
import json
from scipy.stats import chisquare, pearsonr, ks_2samp
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [22]:
# Cargar los datos
df = pd.read_excel("../2. data/0 TimeLog (original, no modificar).xlsx")

In [23]:
# Copio para no modificar el original
tl = df.copy()
# Agrego LOS a cada fila
tl["LOS"] = (tl["TF"] - tl["TI"])

In [24]:
def calculate_kde_metrics(y_true, kde):
    
    # Calculo las métricas para el kde (que es el unico que voy a usar)
    def get_metrics(y_true, y_pred):
        # prediccion = y_pred * (y_true.sum() / y_pred.sum())  # reescala predicciones para el test
        # chi2 = chisquare(f_obs=y_true, f_exp=prediccion)
        chi2 = chisquare(f_obs=y_true, f_exp=y_pred)
        ks_stat = ks_2samp(y_true, y_pred, method = "asymp")
        cc = pearsonr(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        mae = mean_absolute_error(y_true, y_pred)

        return {
            'Chi2': {"Value": chi2.statistic, "p-value": chi2.pvalue},
            'KS': {"Value": ks_stat.statistic, "p-value": ks_stat.pvalue},
            'CC': {"Value": cc.statistic, "p-value": cc.pvalue},
            'R2': {"Value": r2, "p-value": "NA"},
            'RMSE': {"Value": rmse, "p-value": "NA"},
            'MAE': {"Value": mae, "p-value": "NA"}
        }
    metrics = get_metrics(y_true, kde * y_true.sum())

    return metrics

# Funciones para KDE Discreto Triangular

# Discrete Triangular Kernel
def D(a1, a2, h):
    left = sum(k**h for k in range(1, a1 + 1))
    right = sum(k**h for k in range(1, a2 + 1))
    return (a1 + a2 + 1) - (a1 + 1)**(-h) * left - (a2 + 1)**(-h) * right

# Discrete Triangular Distribution Denominator Function
def DT_kernel(y, m, a1, a2, h):
    denom = D(a1, a2, h)
    if y < m - a1 or y > m + a2:
        return 0.0
    if y < m:
        return (1 - ((m - y) / (a1 + 1))**h) / denom
    elif y > m:
        return (1 - ((y - m) / (a2 + 1))**h) / denom
    else:  # y == m
        return 1 / denom

# KDE using the discrete triangular kernel
def discrete_DT_kde(data, a1=2, a2=2, h=1.0, support=None):
    if support is None:
        support = np.arange(min(data), max(data) + 1)
    kde_pmf = np.zeros_like(support, dtype=float)
    for x in data:
        for i, y in enumerate(support):
            kde_pmf[i] += DT_kernel(y, x, a1, a2, h)
    kde_pmf /= kde_pmf.sum()  # Normalize to make it a valid PMF
    return support, kde_pmf

# Search for the smallest h with p > 0.065
def find_satisfactory_h(data, empirical_counts, support, a1=1, a2=1, initial_h=0.5, p_lb=0.065, p_ub=0.1):
    p = 0
    h = initial_h
    contador = 0
    while True:
        _, kde = discrete_DT_kde(data, a1=a1, a2=a2, h=h, support=support)
        expected_counts = kde * empirical_counts.sum()
        chi2, p = chisquare(empirical_counts, expected_counts)
        if p > p_lb and p < p_ub:
            return h, kde, chi2, p
        elif p > p_ub:
            h += 0.01
        elif p < p_lb:
            h -= 0.01
        # Check if h is too small
        if h < 0.02:
            return None, None, None, None
        contador += 1
        if contador > 100:
            return h, kde, chi2, p

# Main function
def ajuste_kde_triangular(tl, unidad, grd, hospital, plot, a1=1, a2=0, initial_h=1, p_lb=0.065, p_ub=0.1):
    # De aqui en adelante se reemplaza en la función
    tl_u = tl[tl["UNIDAD"].isin(["ICU", "SDU_WARD"])]
    v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{hospital}")]
    vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
    vector.columns = ["LOS", "count"]
    vector["LOS"] = vector["LOS"] // 12

    # Reindexamos para que todo el soporte tenga al menos un valor llenando los ceros con 1
    values = np.arange(int(min(vector["LOS"])), int(max(vector["LOS"])) + 1) # Soporte para el KDE
    vector_filled = vector.set_index("LOS").reindex(values, fill_value=1).reset_index()
    empirical_counts = vector_filled["count"].values
    los_samples = np.repeat(vector_filled["LOS"], vector_filled["count"])
    empirical_pmf = empirical_counts / empirical_counts.sum()

    # Probar con diferentes valores de h
    best_h, final_kde_pmf, chi2, p_val = find_satisfactory_h(los_samples, empirical_counts, values, a1=a1, a2=a2, initial_h=initial_h, p_lb=p_lb, p_ub=p_ub)

    if best_h is not None:
        if plot:
            # Plot the KDE vs empirical PMF
            plt.figure(figsize=(10, 6))
            plt.bar(values - 0.2, empirical_pmf, width=0.4, label="Empirical PMF", alpha=0.7)
            plt.bar(values + 0.2, final_kde_pmf, width=0.4, label=f"DT KDE (h={best_h:.2f})", alpha=0.7)
            plt.title("LOS Empirico vs. KDE Discreto Triangular")
            plt.xlabel("LOS")
            plt.ylabel("Probabilidad")
            plt.xticks(values)
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            folder = "plots/los"
            filename = f"hospital {hospital}, {unidad}, grd: {grd}, h: {best_h:.2f}, Chi-2: {chi2:.4f}, P-value: {p_val:.4f}.png"
            os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist
            path = os.path.join(folder, filename)
            plt.savefig(path, format='png', dpi=300)
            plt.close()
            

        print(f"hospital {hospital}, {unidad}, grd: {grd}, h: {best_h:.2f}, Chi-2: {chi2:.4f}, P-value: {p_val:.4f}")
        return best_h, final_kde_pmf, chi2, p_val
    else:
        print(f"hospital {hospital}, {unidad}, grd: {grd}, No value of h produced a KDE with p-value > 0.065")

In [25]:
def empirical_counts(unidad, grd, hospital, tl=tl):
    # De aqui en adelante se reemplaza en la función
    tl_u = tl[tl["UNIDAD"].isin(["ICU", "SDU_WARD"])]
    v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{hospital}")]
    vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
    vector.columns = ["LOS", "count"]
    vector["LOS"] = vector["LOS"] // 12
    # Reindexamos para que todo el soporte tenga al menos un valor llenando los ceros con 1
    values = np.arange(int(min(vector["LOS"])), int(max(vector["LOS"])) + 1) # Soporte para el KDE
    vector_filled = vector.set_index("LOS").reindex(values, fill_value=1).reset_index()
    empirical_counts = vector_filled["count"].values
    return empirical_counts

plot = True
resultados = {}
# Separando por hospital
for hospital in range(1, 4):
    resultados[hospital] = {}
    for unidad in ["ICU", "SDU_WARD"]:
        resultados[hospital][unidad] = {}
        for grd in range(1, 9):
            best_h, final_kde_pmf, chi2, p_val = ajuste_kde_triangular(tl, unidad, grd, hospital, plot, a1=1, a2=0, initial_h=1, p_lb=0.065, p_ub=0.09)
            resultados[hospital][unidad][grd] = {
                "best_h": best_h,
                "final_kde_pmf": final_kde_pmf.tolist(),
                "metricas": calculate_kde_metrics(empirical_counts(unidad, grd, hospital), final_kde_pmf)
            }

hospital 1, ICU, grd: 1, h: 0.87, Chi-2: 65.8364, P-value: 0.0659
hospital 1, ICU, grd: 2, h: 0.93, Chi-2: 70.2872, P-value: 0.0674
hospital 1, ICU, grd: 3, h: 0.89, Chi-2: 39.9401, P-value: 0.0669
hospital 1, ICU, grd: 4, h: 0.80, Chi-2: 64.3247, P-value: 0.0699
hospital 1, ICU, grd: 5, h: 0.27, Chi-2: 50.5026, P-value: 0.0685
hospital 1, ICU, grd: 6, h: 0.30, Chi-2: 25.6121, P-value: 0.0818
hospital 1, ICU, grd: 7, h: 0.85, Chi-2: 70.2350, P-value: 0.0680
hospital 1, ICU, grd: 8, h: 1.00, Chi-2: 50.8652, P-value: 0.0792
hospital 1, SDU_WARD, grd: 1, h: 0.74, Chi-2: 64.1770, P-value: 0.0716
hospital 1, SDU_WARD, grd: 2, h: 1.05, Chi-2: 64.3009, P-value: 0.0841
hospital 1, SDU_WARD, grd: 3, h: 0.53, Chi-2: 51.8693, P-value: 0.0662
hospital 1, SDU_WARD, grd: 4, h: 1.34, Chi-2: 65.0694, P-value: 0.0890
hospital 1, SDU_WARD, grd: 5, h: 0.61, Chi-2: 68.1711, P-value: 0.0655
hospital 1, SDU_WARD, grd: 6, h: 0.71, Chi-2: 49.1674, P-value: 0.0706
hospital 1, SDU_WARD, grd: 7, h: 0.70, Chi-2: 

In [26]:
def save_dict_as_json(data_dict, filename, folder):
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist
    path = os.path.join(folder, filename)
    with open(path, 'w') as f:
        json.dump(data_dict, f, indent=4)
    print(f"Dictionary saved to: {path}")

In [27]:
# Save the results to a JSON file
save_dict_as_json(resultados, filename="los.json", folder="resultados incertidumbre")

Dictionary saved to: resultados incertidumbre/los.json
